In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install py4j

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-05 23:37:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-05 23:37:25 (6.63 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

### Create DataFrames to match tables

In [10]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).drop_duplicates()
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1007WCO9ANK5A|   51064783|B00000J637|     392532841| 1999-12-19|
|R1008C0UCJ947Z|   38214553|B0030CAZ46|     838006014| 2013-11-03|
|R10090GQP2KUGC|   40683803|B000J2344S|     733108562| 2009-03-16|
|R100BU9DTXGV52|   17151713|B005LS4N22|     720938440| 2014-12-06|
|R100CYT259Z5F6|   33020139|B000002JX4|     170184461| 2005-04-24|
|R100LBB4I86SV7|   37027124|B000002HRE|      23284524| 2011-10-05|
|R100P2CJLMV1GG|   15271496|B002LFAHD2|      30974421| 2009-10-17|
|R100WS6KLLYOK9|   35475809|B009PWN13M|     107873959| 2015-03-03|
|R100YMV9SCG3H7|   47088492|B000063CNC|     752469199| 2015-03-20|
|R101LOQDMJNPC1|   50719501|B0000025CB|     543611659| 2000-04-04|
|R101OHRJHTHWO8|   31932849|B00000AFGN|     712988630| 2009-11-18|
|R101QN85X3SRAC|   50620937|B000003H1D|     769358950| 2000-06

In [19]:
# Create the customers_table DataFrame
from pyspark.sql.functions import count
customers_df = review_df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45184767|             9|
|   15324792|            17|
|   35442596|             4|
|   12862918|             1|
|   42793139|             4|
|   25485327|             1|
|   40728070|           242|
|   14312164|            10|
|    9137250|            20|
|   43560898|            23|
|   38490977|             1|
|   39430203|           113|
|   19324145|            33|
|   39767518|           171|
|   30682568|            58|
|   35939026|            43|
|   50733545|           133|
|   14502539|             1|
|   50064220|             5|
|   53067883|            24|
+-----------+--------------+
only showing top 20 rows



In [29]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"])
products_df = products_df.drop_duplicates()
products_df.show(truncate=False)

+----------+----------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                 |
+----------+----------------------------------------------------------------------------------------------+
|B00TXH4OLC|Whatever's for Us: Remastered                                                                 |
|B00B6QXN6U|Same Trailer Different Park                                                                   |
|B001GCZXW6|Soaring (Jazz Club)                                                                           |
|B000003EK6|CARIBBEAN PARTY                                                                               |
|B00N1F0BKK|Pain Killer                                                                                   |
|B00V7KAO7Q|A Thoughtiverse Unmarred                                                                      |
|B000094Q4P|Comfort of Her W

In [20]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3LI5TRP3YIDQL|   10140119|B00TXH4OLC|     384427924| 2015-08-31|
|R3LGC3EKEG84PX|   27664622|B00B6QXN6U|     831769051| 2015-08-31|
| R9PYL3OYH55QY|   45946560|B001GCZXW6|      14067376| 2015-08-31|
|R3PWBAWUS4NT0Q|   15146326|B000003EK6|     566295619| 2015-08-31|
|R15LYP3O51UU9E|   16794688|B00N1F0BKK|     210426072| 2015-08-31|
|R1AD7L0CC3DSRI|   32203364|B00V7KAO7Q|     360249504| 2015-08-31|
|R32FE8Y45QV434|    1194276|B000094Q4P|      20972283| 2015-08-31|
|R3NM4MZ4XWL43Q|   45813052|B00JMK0P1I|     378049560| 2015-08-31|
|R3H4FXX6Q7I37D|   12795687|B008OW1S3O|     141620702| 2015-08-31|
|R30L5PET7LFFDC|   36673840|B00VI2L3L4|     920557135| 2015-08-31|
|  REFRE1LEKLAF|   49453576|B0000041EV|     683128964| 2015-08-31|
|R3JTJ5EQN74E9H|    3285047|B00005YW4H|     268461622| 2015-08

In [21]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [24]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:<endpoint>:5432/<database>"
config = {"user":"postgres", 
          "password": <password>, 
          "driver":"org.postgresql.Driver"}

In [25]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [27]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [28]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)